In [2]:
import os
import time
import pandas as pd
import numpy as np
import numpy.random as npr
import copy
import re
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from tqdm import tqdm
from scipy.special import digamma, loggamma
from scipy.sparse import csr_matrix
from typing import List, Dict, Tuple, Set, Optional

In [16]:
def log_sum_exp(vec):
    vec_max = np.max(vec, axis=0)
    exp_vec = np.exp(vec - vec_max)
    sum_exp_vec = np.sum(exp_vec)
    log_sum_exp = np.log(sum_exp_vec) + vec_max
    return log_sum_exp

def init_variational_params(documents, K, rs_int=npr.randint(low=0, high=100)):
    rs = npr.RandomState(rs_int)
    N, V = documents.shape
    LAMBDA = rs.uniform(low=0.1, high=1.0, size=(K, V))
    GAMMA = rs.uniform(low=0.1, high=1.0, size=(N, K))
    PHI = []
    for document in documents:
        M = np.sum((document > 0).astype("int32"))
        document_PHI = np.ones((M, K))
        document_PHI = document_PHI / K
        PHI.append(document_PHI)
        
    return LAMBDA, GAMMA, PHI

def compute_ELBO(LAMBDA, GAMMA, PHI, documents, nonzero_idxs, K):
    ELBO = 0
    N, _ = documents.shape

    E_log_p_BETA = np.sum((ETA-1) * (digamma(LAMBDA) - digamma(np.sum(LAMBDA, axis=1, keepdims=True))))
    ELBO += E_log_p_BETA

    E_log_p_THETA = np.sum((ALPHA-1) * (digamma(GAMMA) - digamma(np.sum(GAMMA, axis=1, keepdims=True))))
    ELBO += E_log_p_THETA

    E_log_p_x_z = 0
    for i in range(N):
        document = documents[i]
        nonzero_idx = nonzero_idxs[i]
        word_idx = 0
        for idx in nonzero_idx:
            E_log_p_x_z += np.sum(PHI[i][word_idx] * (digamma(GAMMA[i])-digamma(np.sum(GAMMA[i])))) \
                + np.sum(PHI[i][word_idx] * (digamma(LAMBDA[:, idx])-digamma(np.sum(LAMBDA, axis=1))))
            word_idx += 1
    ELBO += E_log_p_x_z

    E_log_q_BETA = np.sum(-loggamma(np.sum(LAMBDA, axis=1)) + np.sum(loggamma(LAMBDA), axis=1) \
        - np.sum((LAMBDA - 1) * (digamma(LAMBDA) - digamma(np.sum(LAMBDA, axis=1, keepdims=True))), axis=1))
    ELBO += E_log_q_BETA

    E_log_q_THETA = np.sum(-loggamma(np.sum(GAMMA, axis=1)) + np.sum(loggamma(GAMMA), axis=1) \
        - np.sum((GAMMA - 1) * (digamma(GAMMA) - digamma(np.sum(GAMMA, axis=1, keepdims=True))), axis=1))
    ELBO += E_log_q_THETA

    E_log_q_z = 0
    for i in range(N):
        document = documents[i]
        nonzero_idx = nonzero_idxs[i]
        word_idx = 0
        for idx in nonzero_idx:
            E_log_q_z += -np.sum(PHI[i][word_idx] * np.log(PHI[i][word_idx]))
            word_idx += 1
    ELBO += E_log_q_z

    return ELBO

In [4]:
def simulate_LDA(N, Ms, K, V, ETA, ALPHA, rs_int=np.random.randint(low=0, high=100)):
    rs = npr.RandomState(rs_int) 
    BETA = rs.dirichlet(np.full(V, ETA), size=K)
    THETA = rs.dirichlet(np.full(K, ALPHA), size=N)
    
    row_idxs = []
    col_idxs = []
    values = []
    nonzero_idxs = []

    for i in range(N):
        doc_word_counts = np.zeros(V)
        for _ in range(Ms[i]):
            z_ij = rs.choice(K, p=THETA[i])
            x_ij = rs.choice(V, p=BETA[z_ij])
            doc_word_counts[x_ij] += 1
        doc_nonzero = np.nonzero(doc_word_counts)[0]
        doc_nonzero = np.array(sorted(doc_nonzero))
        nonzero_idxs.append(doc_nonzero)

        row_idxs.extend([i] * len(doc_nonzero))
        col_idxs.extend(doc_nonzero)
        values.extend(doc_word_counts[doc_nonzero])
    documents = csr_matrix((values, (row_idxs, col_idxs)), shape=(N, V)).toarray()
    
    return documents, nonzero_idxs, BETA, THETA

In [7]:
N = 100
Ms = npr.poisson(200, size=N)
K = 10
V = 1000
ETA = 100 / V
ALPHA = 1 / K
documents, nonzero_idxs, BETA, THETA = simulate_LDA(N, Ms, K, V, ETA, ALPHA)
LAMBDA, GAMMA, PHI = init_variational_params(documents, K)

In [407]:
rs = npr.RandomState(0)
N = 10
K = 5
V = 100
Ms = rs.poisson(50, size=N)
eta0 = 0.3
alpha0 = 0.5
X, nonzero_idxs, _, _ = simulate_LDA(N, Ms, K, V, eta0, alpha0, rs_int=0)
lambd, gamma, phi = init_variational_params(X, K, rs_int=0)
E_log_q_z = 0
for i in range(N):
    document = documents[i]
    nonzero_idx = nonzero_idxs[i]
    word_idx = 0
    for idx in nonzero_idx:
        E_log_q_z += -document[idx] * np.sum(phi[i][word_idx] * np.log(phi[i][word_idx]))
        word_idx += 1
E_log_q_z

np.float64(41.84538572328661)

In [8]:
N = 100
Ms = npr.poisson(100, size=N)
K = 10
V = 1000
ETA = 100 / V
ALPHA = 1 / K
documents, nonzero_idxs, BETA, THETA = simulate_LDA(N, Ms, K, V, ETA, ALPHA)
LAMBDA, GAMMA, PHI = init_variational_params(documents, K)

In [ ]:
start = time.time()
ELBOs = []
prev_ELBO = -np.inf
curr_ELBO = compute_ELBO(LAMBDA, GAMMA, PHI, documents, nonzero_idxs, K)
ELBOs.append(curr_ELBO)
print(f"Initial ELBO: {ELBOs[0]}\n")

max_iterations = 200
tol = 10e-4
LAMBDA_t = copy.deepcopy(LAMBDA)
GAMMA_t = copy.deepcopy(GAMMA)
PHI_t = copy.deepcopy(PHI)

for t in range(max_iterations):
    print(f"Iteration {t+1}")
    for i in tqdm(range(N), desc="Updating PHI and GAMMA"):
        document = documents[i]
        nonzero_idx = nonzero_idxs[i]
        GAMMA_i_t = copy.deepcopy(GAMMA_t[i])
        word_idx = 0
        for idx in nonzero_idx:
            log_PHI_ij = np.zeros((K,))
            for k in range(K):
                LAMBDA_k_t = copy.deepcopy(LAMBDA_t[k])
                exp_propto = digamma(GAMMA_i_t[k]) - digamma(np.sum(GAMMA_i_t)) + digamma(LAMBDA_k_t[idx]) - digamma(np.sum(LAMBDA_k_t))
                log_PHI_ij[k] = exp_propto
            PHI_ij = np.exp(log_PHI_ij - log_sum_exp(log_PHI_ij))
            PHI_t[i][word_idx] = PHI_ij
            word_idx += 1
        GAMMA_i_t = np.zeros((K,)) + ALPHA
        for k in range(K):
            GAMMA_i_t[k] += np.sum(document[nonzero_idx] * PHI_t[i][:, k])
        GAMMA_t[i] = GAMMA_i_t

    for k in tqdm(range(K), desc="Updating LAMBDA"):
        LAMBDA_k_t = np.zeros((V,)) + ETA
        for i in range(N):
            document = documents[i]
            nonzero_idx = nonzero_idxs[i]
            word_idx = 0
            for idx in nonzero_idx:
                LAMBDA_k_t[idx] += document[idx] * PHI_t[i][word_idx][k]
                word_idx += 1
            LAMBDA_t[k] = LAMBDA_k_t

    prev_ELBO = curr_ELBO
    curr_ELBO = compute_ELBO(LAMBDA_t, GAMMA_t, PHI_t, documents, nonzero_idxs, K)
    ELBOs.append(curr_ELBO)
    print(f"Current ELBO: {curr_ELBO} | Change in ELBO: {curr_ELBO - prev_ELBO}\n")

    if abs(curr_ELBO - prev_ELBO) < tol:
        break
stop = time.time()

LAMBDA_final = copy.deepcopy(LAMBDA_t)
GAMMA_final = copy.deepcopy(GAMMA_t)
PHI_final = copy.deepcopy(PHI_t)

plt.ticklabel_format(style="sci", axis="y", scilimits=(0, 0))
plt.plot(np.linspace(0, stop-start, len(ELBOs)), ELBOs)

In [23]:
rs = npr.RandomState(0)
K, V, N = 10, 300, 30
eta0, alpha0 = 0.1, (50 / K)
Ms = rs.poisson(60, size=N)
documents, nonzero_idxs, BETA, THETA = simulate_LDA(N, Ms, K, V, eta0, alpha0, 0)
lambd, gamma, phi = init_variational_params(documents, K, 0)
compute_ELBO(lambd, gamma, phi, documents, nonzero_idxs, K)

np.float64(-9416.708794025422)

In [12]:
LAMBDA_final

array([[0.10002374, 0.10000619, 0.10012869, ..., 0.10005261, 2.67731004,
        0.10004352],
       [0.10000678, 0.10000381, 0.10004467, ..., 0.10001745, 0.10002651,
        0.10001298],
       [0.10006586, 0.10001275, 0.10004989, ..., 0.10002568, 0.10000259,
        0.10001517],
       ...,
       [5.09978464, 0.10000621, 0.10007385, ..., 0.10004511, 0.10002148,
        0.10000869],
       [0.10003886, 0.10001988, 2.09953683, ..., 7.49199056, 0.10002397,
        0.10001101],
       [0.10002628, 0.1000771 , 0.10003458, ..., 0.10001568, 0.100024  ,
        0.1000269 ]])